In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

#import symbolic math liibbrary
import sympy as sp


In [2]:
def prep_Sensor_unnorm(P, X1, X2, X3):
    """
    Takes in a nozero position and sensors positions ad calculates a normalized version with the unit vectors to the sensors
    """

    # Calculate the unit vectors to the sensors
    r1 = np.array([X1[0] - P[0], X1[1] - P[1], X1[2] - P[2]])
    r2 = np.array([X2[0] - P[0], X2[1] - P[1], X2[2] - P[2]])
    r3 = np.array([X3[0] - P[0], X3[1] - P[1], X3[2] - P[2]])

    # Normalize the vectors
    r1 /= np.linalg.norm(r1)
    r2 /= np.linalg.norm(r2)
    r3 /= np.linalg.norm(r3)

    return r1, r2, r3





In [3]:
    
def Q_simplfied(X1, X2, X3):
        
    X1 = np.vstack((X1))
    X2 = np.vstack((X2))
    X3 = np.vstack((X3))

    G_GT = X1@X1.T + X2@X2.T + X3@X3.T

    G_GT_inv = np.linalg.inv(G_GT)

    HDOP = np.sqrt(G_GT_inv[0, 0]+ G_GT_inv[1, 1])

    return G_GT_inv, HDOP


In [4]:
def Q_analytical(P, X1, X2, X3):
    sensor = np.array([X1, X2, X3])
    G = []

    for i in range(len(sensor)):
        delt = sensor[i] - P
        norm = np.linalg.norm(delt)
        G.append(delt/norm)
    G = np.array(G) 

    G_GT = G.T @ G
    G_GT_inv = np.linalg.inv(G_GT)
    HDOP = np.sqrt(G_GT_inv[0, 0] + G_GT_inv[1, 1])
    return G_GT_inv, HDOP

In [5]:
def Q_paper(P, X1, X2, X3):
    """
    Takes in a position and sensors positions and calculates the Q matrix and HDOP value
    """
    I = np.identity(3)

    X1 = np.vstack((X1))
    X2 = np.vstack((X2))
    X3 = np.vstack((X3))


    P_x_1 = I - (X1 @ X1.T)/ (X1.T @ X1)
    P_x_2 = I - (X2 @ X2.T)/ (X2.T @ X2)
    P_x_3 = I - (X3 @ X3.T)/ (X3.T @ X3)

    P_x = P_x_1 + P_x_2 + P_x_3

    G_GT_inv = np.linalg.inv(P_x)
    HDOP = np.sqrt(G_GT_inv[0, 0] + G_GT_inv[1, 1])


    return G_GT_inv, HDOP

Lets explain a Position Estimation and timing system


In [6]:
def Q_comparison(P, X1, X2, X3, r1, r2, r3):
    Q_Ana, HDOP_Ana = Q_analytical(P, X1, X2, X3)
    Q_Sim, HDOP_Sim= Q_simplfied(r1, r2, r3)
    Q_pap, HDOP_pap= Q_paper(P, r1, r2, r3)

    #print("Q_analytical:" , Q_Ana)
    #print("Q_simplified:" , Q_Sim)
    print("HDOP_analytical:" , HDOP_Ana)
    print("HDOP_simplified:" , HDOP_Sim)
    print("Q_paper:" , HDOP_pap)
    #print("Difference in HDOP: ", HDOP_Ana - HDOP_Sim)

    return HDOP_Ana- HDOP_Sim

In [107]:
#Initial values
P = np.array([5.0, 5.0, 0.0])

X1 = np.array([10, 20, 10])
X2 = np.array([20, 10, 10])
X3 = np.array([10, 10, 20])




#sensror values
#X1 = np.array([.707, 0.707, 0.0])
#X2 = np.array([0.0, 1.0, 0.0])

def X3_func(X3, x=.1, y=.1, z=.1):
    X3 = np.array([X3[0] + x, X3[1] + y, X3[2] +z])
    return X3 / np.linalg.norm(X3)







In [108]:
for i in range(10):
    X3 = X3_func(X3, x=1, y=0, z=0)
    #print("X3:", X3)
    r1, r2, r3 = prep_Sensor_unnorm(P, X1, X2, X3)
    error = Q_comparison(P, X1, X2, X3, r1, r2, r3)
    print("error at iteration", i, ":", error)

HDOP_analytical: 2.0626693881350726
HDOP_simplified: 2.062669388135073
Q_paper: 1.3582908456746463
error at iteration 0 : -4.440892098500626e-16
HDOP_analytical: 2.0850527469767326
HDOP_simplified: 2.0850527469767326
Q_paper: 1.387116454955761
error at iteration 1 : 0.0
HDOP_analytical: 2.1065911711001784
HDOP_simplified: 2.106591171100179
Q_paper: 1.408412338277138
error at iteration 2 : -4.440892098500626e-16
HDOP_analytical: 2.1198117374262164
HDOP_simplified: 2.119811737426217
Q_paper: 1.42006747068063
error at iteration 3 : -4.440892098500626e-16
HDOP_analytical: 2.1269452607238866
HDOP_simplified: 2.1269452607238875
Q_paper: 1.426036363608727
error at iteration 4 : -8.881784197001252e-16
HDOP_analytical: 2.1306258852972837
HDOP_simplified: 2.1306258852972833
Q_paper: 1.4290417037769183
error at iteration 5 : 4.440892098500626e-16
HDOP_analytical: 2.1324921705506075
HDOP_simplified: 2.1324921705506084
Q_paper: 1.4305477772030748
error at iteration 6 : -8.881784197001252e-16
HDOP_a